In [1]:
import pandas as pd
import requests
import os
pd.options.display.max_rows = 5
pd.options.display.max_columns = None

Creamos la funcion "anho", la cual nos retornara un dataframe con los avistamientos de aves registrados en tal año en Chile

In [5]:
def anho(y):
    print(" ")
    key = "otqvshn22qrt"
    regionCode = "CL"
    payload={}
    headers = {
        'X-eBirdApiToken': key
    }
    df = pd.DataFrame()
    for m in range(1, 13):
        print(f"{m}-{y}", end=" ; ")
        for d in range(1, 32):
            url = url = f"https://api.ebird.org/v2/data/obs/{regionCode}/historic/{y}/{m}/{d}"
            response = requests.request("GET", url, headers=headers, data=payload)
            df = pd.concat([df, pd.json_normalize(response.json())], ignore_index=True)
    return df

Principalmente nos interesan los años desde el 2015, hasta el 2022. Estos dataframes, los guaradremos en archivos csv.

In [6]:
for y in range(2015, 2023):
    df = anho(y)
    df.to_csv(f"vis_{y}.csv", index=False)

 
1-2015 ; 2-2015 ; 3-2015 ; 4-2015 ; 5-2015 ; 6-2015 ; 7-2015 ; 8-2015 ; 9-2015 ; 10-2015 ; 11-2015 ; 12-2015 ;  
1-2016 ; 2-2016 ; 3-2016 ; 4-2016 ; 5-2016 ; 6-2016 ; 7-2016 ; 8-2016 ; 9-2016 ; 10-2016 ; 11-2016 ; 12-2016 ;  
1-2017 ; 2-2017 ; 3-2017 ; 4-2017 ; 5-2017 ; 6-2017 ; 7-2017 ; 8-2017 ; 9-2017 ; 10-2017 ; 11-2017 ; 12-2017 ;  
1-2018 ; 2-2018 ; 3-2018 ; 4-2018 ; 5-2018 ; 6-2018 ; 7-2018 ; 8-2018 ; 9-2018 ; 10-2018 ; 11-2018 ; 12-2018 ;  
1-2021 ; 2-2021 ; 3-2021 ; 4-2021 ; 5-2021 ; 6-2021 ; 7-2021 ; 8-2021 ; 9-2021 ; 10-2021 ; 11-2021 ; 12-2021 ;  
1-2022 ; 2-2022 ; 3-2022 ; 4-2022 ; 5-2022 ; 6-2022 ; 7-2022 ; 8-2022 ; 9-2022 ; 10-2022 ; 11-2022 ; 12-2022 ; 

Leemos uno de los archivos

In [8]:
df_2019 = pd.read_csv("Avistamientos/vis_2019.csv")
#df_2020 = pd.read_csv("vis_2020.csv")

In [69]:
df_2019

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId,exoticCategory,errors
0,soulap1,Southern Lapwing,Vanellus chilensis,L4026975,Loma Linda,2019-01-01 23:00,1.0,-33.435764,-71.677103,True,False,False,S51197645,NaN,NaN
1,brnowl,Barn Owl,Tyto alba,L4026975,Loma Linda,2019-01-01 23:00,1.0,-33.435764,-71.677103,True,False,False,S51197645,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62679,chtsee1,Chestnut-throated Seedeater,Sporophila telasco,L760571,Azapa,2019-12-31 08:10,10.0,-18.517247,-70.197006,True,False,False,S63139126,NaN,NaN
62680,perter2,Peruvian Tern,Sternula lorata,L12031093,Hornitos,2019-12-31 07:20,1.0,-22.872257,-70.288054,True,True,True,S72204442,NaN,NaN


Creamos el df ubicaciones, el cual solo tendra la informacion de los lugares, luego eliminamos los registros donde no se haya datado el lugar, y posteriormente agrupamos por locName, de esta manera disminuimos la cantidad de peticiones que haremos mas adelante para obtener la region de estas ubicaciones

In [2]:
ubicaciones = pd.DataFrame()
for archivo in os.listdir("Avistamientos"):
    if archivo.endswith(".csv"):
        df = pd.read_csv(f"Avistamientos/{archivo}")
        ubicaciones = pd.concat([ubicaciones, df], ignore_index=True)
ubicaciones = ubicaciones[["locId","locName"]]
ubicaciones = ubicaciones.drop_duplicates(["locName"], keep='first')
ubicaciones

C:\Users\matar\AppData\Local\Temp\ipykernel_1912\3427554421.py:4: DtypeWarning: Columns (9,10,11,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"Avistamientos/{archivo}")


,locId,locName
0,L3247303,phm
4,L2973546,Desembocadura del estero culebron
...,...,...
457370,L22016485,Cruce Ruta 9 - Cueva del Milodón - Cruce Y-150...
457371,L21949753,"Ruta 9, Puerto Zenteno, Magallanes y la Antárt..."


Creamos la funcion region, la cual nos retornara la region de Chile donde ocurrio el avistamiento

In [3]:
def region(locId):
    try:
        key = "otqvshn22qrt"
        payload={}
        headers = {'X-eBirdApiToken': key}
        url = f"https://api.ebird.org/v2/ref/region/info/{locId}"
        response = requests.request("GET", url, headers=headers, data=payload)
        df = pd.json_normalize(response.json())
        region = df["parent.parent.result"][0].split(",")[0]
        return region
    except:
        return None   

Al df ubicaciones, le agregamos la columna region

In [4]:
ubicaciones['region'] = ubicaciones.apply(lambda x: region(x["locId"]), axis=1)

De este df eliminamos las columnas que ya no nos interesan

In [6]:
ubicaciones.drop(["locId"], axis=1, inplace=True)

Guardamos el df en un csv, ya que las peticiones demoraron 474 minutos, y no queremos perder esta valiosa informacion

In [11]:
ubicaciones.to_csv("regiones.csv", index=False)

Leemos el archivo guardado, para no volver a realizar las peticiones

In [12]:
ubicaciones = pd.read_csv("regiones.csv")
ubicaciones

,locName,region
0,phm,Coquimbo
1,Desembocadura del estero culebron,Coquimbo
...,...,...
38137,Cruce Ruta 9 - Cueva del Milodón - Cruce Y-150...,Magallanes
38138,"Ruta 9, Puerto Zenteno, Magallanes y la Antárt...",Magallanes


Unimos las regiones con cada df de los avistamientos

In [13]:
os.makedirs(f"Avistamientos_Region", exist_ok=True)
for archivo in os.listdir("Avistamientos"):
    if archivo.endswith(".csv"):
        df = pd.read_csv(f"Avistamientos/{archivo}")
        df = pd.merge(df, ubicaciones, on="locName", how="left")
        df.dropna(subset=["region"], inplace=True)
        df.to_csv(f"Avistamientos_Region/{archivo}", index=False)
#df_2019 = pd.merge(df_2019, ubicaciones, on="locName", how="left")
#df_2019.dropna(subset=["region"], inplace=True)
#df_2019

C:\Users\matar\AppData\Local\Temp\ipykernel_1912\75000220.py:4: DtypeWarning: Columns (9,10,11,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"Avistamientos/{archivo}")


In [19]:
df = pd.read_csv(f"Avistamientos/vis_2015.csv")
df = pd.merge(df, ubicaciones, on="locName", how="left")
df

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId,exoticCategory,errors,region
0,whshil1,White-sided Hillstar,Oreotrochilus leucopleurus,L3247303,phm,2015-01-01 20:30,1.0,-31.784164,-70.536299,True,False,True,S21135098,NaN,NaN,Coquimbo
1,bkbsht1,Black-billed Shrike-Tyrant,Agriornis montanus,L3247303,phm,2015-01-01 20:30,1.0,-31.784164,-70.536299,True,False,True,S21135098,NaN,NaN,Coquimbo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44035,sctear1,Scale-throated Earthcreeper,Upucerthia dumetaria,L1030752,Pampa Larga,2015-12-31,1.0,-52.267947,-69.732113,True,False,False,S129511082,NaN,NaN,Magallanes
44036,chvtyr2,Chocolate-vented Tyrant,Neoxolmis rufiventris,L1030752,Pampa Larga,2015-12-31,2.0,-52.267947,-69.732113,True,False,False,S129511082,NaN,NaN,Magallanes


In [20]:
none = df[df["region"].isna()]
none

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId,exoticCategory,errors,region
8456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'status': '500', 'code': 'http.status.500', ...",NaN
8457,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'status': '500', 'code': 'http.status.500', ...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43393,humpen1,Humboldt Penguin,Spheniscus humboldti,L781499,Playa Negra,2015-12-27 15:30,1.0,-32.926409,-71.528000,True,False,False,S26477414,NaN,NaN,NaN
43739,greyel,Greater Yellowlegs,Tringa melanoleuca,L3279728,Río Gamboa--desembocadura,2015-12-29,NaN,-42.484789,-73.769867,True,False,False,S34238407,NaN,NaN,NaN
